**1.Download S&P 500 Data**

In [2]:
import pandas as pd
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta as ta
import warnings
warnings.filterwarnings('ignore')



In [3]:
sp500=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date= '2024-09-30'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*10)
print(start_date)

2014-10-03 00:00:00


In [4]:
df= yf.download(tickers=symbols_list,start=start_date,end=end_date).stack()


[*********************100%***********************]  503 of 503 completed


working with all these huge rows and colums data would be inefficient so we would use df.stack()

In [5]:

df.index.names=['date','ticker']
df.columns=df.columns.str.lower()
df

Price                              adj close       close        high  \
date                      ticker                                       
2014-10-03 00:00:00+00:00 A        37.526653   40.708157   40.779686   
                          AAPL     22.089027   24.905001   25.052500   
                          ABBV     38.515572   58.689999   58.779999   
                          ABT      34.355881   41.790001   41.860001   
                          ACGL     18.469999   18.469999   18.490000   
...                                      ...         ...         ...   
2024-09-27 00:00:00+00:00 XYL     134.509995  134.509995  136.289993   
                          YUM     139.919998  139.919998  140.020004   
                          ZBH     108.220001  108.220001  110.190002   
                          ZBRA    368.600006  368.600006  374.029999   
                          ZTS     194.349457  194.809998  197.509995   

Price                                    low        open       volume  
date                      ticker                                       
2014-10-03 00:00:00+00:00 A        40.171673   40.221745    2205065.0  
                          AAPL     24.760000   24.860001  173878400.0  
                          ABBV     57.340000   57.419998   11675600.0  
                          ABT      41.279999   41.410000    3430900.0  
                          ACGL     18.320000   18.333332    1470300.0  
...                                      ...         ...          ...  
2024-09-27 00:00:00+00:00 XYL     133.779999  134.660004     691300.0  
                          YUM     136.990005  137.520004    2778300.0  
                          ZBH     108.190002  108.769997    1166200.0  
                          ZBRA    368.269989  372.299988     213600.0  
                          ZTS     194.419998  195.649994    1343700.0  

[1225998 rows x 6 columns]

<!-- # ### # ###### 2.CALCULATE FEATURES AND TECHNICAL INDICATORS FOR EACH STOCK
1**) garman-klass volatility
2)RSI
3)Bollinger bands
4)ATR
5)MACD
6)Dollar volume**
GK = sqrt((0.5 * (log(high / low))^2) - ((2 * log(2) - 1) * (log(close / open))^2))(garman klass simplified in code)

 -->


In [6]:
# **** GARMAN-KLASS VOLATILITY****

df['garman_klass_vol']=((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*(np.log(df['adj close'])-(np.log(df['open'])))**2
df

Price                              adj close       close        high  \
date                      ticker                                       
2014-10-03 00:00:00+00:00 A        37.526653   40.708157   40.779686   
                          AAPL     22.089027   24.905001   25.052500   
                          ABBV     38.515572   58.689999   58.779999   
                          ABT      34.355881   41.790001   41.860001   
                          ACGL     18.469999   18.469999   18.490000   
...                                      ...         ...         ...   
2024-09-27 00:00:00+00:00 XYL     134.509995  134.509995  136.289993   
                          YUM     139.919998  139.919998  140.020004   
                          ZBH     108.220001  108.220001  110.190002   
                          ZBRA    368.600006  368.600006  374.029999   
                          ZTS     194.349457  194.809998  197.509995   

Price                                    low        open       volume  \
date                      ticker                                        
2014-10-03 00:00:00+00:00 A        40.171673   40.221745    2205065.0   
                          AAPL     24.760000   24.860001  173878400.0   
                          ABBV     57.340000   57.419998   11675600.0   
                          ABT      41.279999   41.410000    3430900.0   
                          ACGL     18.320000   18.333332    1470300.0   
...                                      ...         ...          ...   
2024-09-27 00:00:00+00:00 XYL     133.779999  134.660004     691300.0   
                          YUM     136.990005  137.520004    2778300.0   
                          ZBH     108.190002  108.769997    1166200.0   
                          ZBRA    368.269989  372.299988     213600.0   
                          ZTS     194.419998  195.649994    1343700.0   

Price                             garman_klass_vol  
date                      ticker                    
2014-10-03 00:00:00+00:00 A              -0.001745  
                          AAPL           -0.005326  
                          ABBV           -0.061293  
                          ABT            -0.013375  
                          ACGL            0.000021  
...                                            ...  
2024-09-27 00:00:00+00:00 XYL             0.000172  
                          YUM             0.000124  
                          ZBH             0.000158  
                          ZBRA            0.000082  
                          ZTS             0.000107  

[1225998 rows x 7 columns]

In [7]:
#  RSI 
# A technical indicator that measures the speed and change of price movements of an asset,
#  such as a stock, index, or ETF. The RSI is displayed on a scale of 0 to 100, 
#  with readings above 70 indicating an overbought asset and readings below 30 indicating an oversold asset

df['rsi']=df.groupby(level=1)['adj close'].transform(lambda x : ta.rsi(close=x , length=20))
# df.xs('AAPL',level=1)['rsi'].plot()(#to ckeck the rsi yes its correct)

In [8]:
# bollinger bandsBollinger Bands are a technical analysis tool used 
# to measure market volatility and identify potential trading opportunities.
# They are made up of three lines: an upper band, a middle band, and a lower band. 
# The middle band is a moving average, while the upper and lower bands are based on standard deviation,
# which is a measure of price volatility.


# Check the output columns of the Bollinger Bands calculation
sample_output = ta.bbands(close=np.log1p(df['adj close']), length=20)
print(sample_output.columns)


Index(['BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0', 'BBB_20_2.0', 'BBP_20_2.0'], dtype='object')


In [9]:
# Calculate the lower Bollinger Band
df['bb_low'] = df.groupby(level=1)['adj close'].transform(
    lambda x: ta.bbands(close=np.log1p(x), length=20)['BBL_20_2.0'] if ta.bbands(close=np.log1p(x), length=20) is not None else np.nan
)

# Calculate the middle Bollinger Band
df['bb_middle'] = df.groupby(level=1)['adj close'].transform(
    lambda x: ta.bbands(close=np.log1p(x), length=20)['BBM_20_2.0'] if ta.bbands(close=np.log1p(x), length=20) is not None else np.nan
)

# Calculate the upper Bollinger Band
df['bb_high'] = df.groupby(level=1)['adj close'].transform(
    lambda x: ta.bbands(close=np.log1p(x), length=20)['BBU_20_2.0'] if ta.bbands(close=np.log1p(x), length=20) is not None else np.nan
)
df

Price                              adj close       close        high  \
date                      ticker                                       
2014-10-03 00:00:00+00:00 A        37.526653   40.708157   40.779686   
                          AAPL     22.089027   24.905001   25.052500   
                          ABBV     38.515572   58.689999   58.779999   
                          ABT      34.355881   41.790001   41.860001   
                          ACGL     18.469999   18.469999   18.490000   
...                                      ...         ...         ...   
2024-09-27 00:00:00+00:00 XYL     134.509995  134.509995  136.289993   
                          YUM     139.919998  139.919998  140.020004   
                          ZBH     108.220001  108.220001  110.190002   
                          ZBRA    368.600006  368.600006  374.029999   
                          ZTS     194.349457  194.809998  197.509995   

Price                                    low        open       volume  \
date                      ticker                                        
2014-10-03 00:00:00+00:00 A        40.171673   40.221745    2205065.0   
                          AAPL     24.760000   24.860001  173878400.0   
                          ABBV     57.340000   57.419998   11675600.0   
                          ABT      41.279999   41.410000    3430900.0   
                          ACGL     18.320000   18.333332    1470300.0   
...                                      ...         ...          ...   
2024-09-27 00:00:00+00:00 XYL     133.779999  134.660004     691300.0   
                          YUM     136.990005  137.520004    2778300.0   
                          ZBH     108.190002  108.769997    1166200.0   
                          ZBRA    368.269989  372.299988     213600.0   
                          ZTS     194.419998  195.649994    1343700.0   

Price                             garman_klass_vol        rsi    bb_low  \
date                      ticker                                          
2014-10-03 00:00:00+00:00 A              -0.001745        NaN       NaN   
                          AAPL           -0.005326        NaN       NaN   
                          ABBV           -0.061293        NaN       NaN   
                          ABT            -0.013375        NaN       NaN   
                          ACGL            0.000021        NaN       NaN   
...                                            ...        ...       ...   
2024-09-27 00:00:00+00:00 XYL             0.000172  52.810998  4.849547   
                          YUM             0.000124  62.536185  4.867805   
                          ZBH             0.000158  48.599342  4.625674   
                          ZBRA            0.000082  64.223784  5.759747   
                          ZTS             0.000107  61.511356  5.218571   

Price                             bb_middle   bb_high  
date                      ticker                       
2014-10-03 00:00:00+00:00 A             NaN       NaN  
                          AAPL          NaN       NaN  
                          ABBV          NaN       NaN  
                          ABT           NaN       NaN  
                          ACGL          NaN       NaN  
...                                     ...       ...  
2024-09-27 00:00:00+00:00 XYL      4.891114  4.932681  
                          YUM      4.902218  4.936632  
                          ZBH      4.687395  4.749115  
                          ZBRA     5.851378  5.943010  
                          ZTS      5.256497  5.294424  

[1225998 rows x 11 columns]

Average True Range (ATR) is a technical analysis tool that measures the volatility of a market by averaging the true ranges over a specified period: 
ATR is a key component of many traders' risk management strategies because it helps them:
Set stop losses
ATR helps traders set more accurate stop-loss levels that account for an asset's natural price fluctuations.
Determine position sizes
ATR helps traders determine position sizes that align with their risk tolerance.
Identify breakouts or trend reversals
Traders can monitor changes in ATR values to identify potential breakouts or trend reversals

In [10]:
#ATR
def compute_atr(stock_data):
    atr = ta.atr(high=stock_data['high'], low=stock_data['low'], close=stock_data['close'], length=14)
    # Check if ATR calculation is successful before proceeding
    if atr is not None:
        return atr.sub(atr.mean()).div(atr.std())
    else:
        return pd.Series([np.nan] * len(stock_data), index=stock_data.index)

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

#atr.sub(atr.mean()): Subtracts the mean of the ATR values from each ATR value, 
# centering the data around zero.
#.div(atr.std()): Divides each centered ATR value by the standard deviation of the ATR values, 
# scaling the data. This standardizes the ATR, which now has a mean of 0 and a standard deviation of 1.




**Moving Average Convergence Divergence (MACD) **is a technical indicator that helps investors identify entry points for buying or selling. It tracks the convergence and divergence of two exponential moving averages (EMAs) to gauge the strength of stock price movement: 
Calculation
The MACD line is calculated by subtracting the 26-period EMA from the 12-period EMA. 
Signal line
A nine-period EMA of the MACD line is plotted on the MACD chart as the signal line. 


In [11]:
#MACD
def compute_macd(close):
    # Calculate MACD
    macd = ta.macd(close=close, length=20)
    
    # Check if MACD calculation is successful
    if macd is not None and not macd.empty:
        macd_values = macd.iloc[:, 0]  # Get the MACD values
        return macd_values.sub(macd_values.mean()).div(macd_values.std())
    else:
        return pd.Series([np.nan] * len(close), index=close.index)

# Apply the MACD computation by grouping by the second level
df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)


In [12]:
df['dollar_volume']=(df['adj close']*df['volume'])/1e6
df

Price                              adj close       close        high  \
date                      ticker                                       
2014-10-03 00:00:00+00:00 A        37.526653   40.708157   40.779686   
                          AAPL     22.089027   24.905001   25.052500   
                          ABBV     38.515572   58.689999   58.779999   
                          ABT      34.355881   41.790001   41.860001   
                          ACGL     18.469999   18.469999   18.490000   
...                                      ...         ...         ...   
2024-09-27 00:00:00+00:00 XYL     134.509995  134.509995  136.289993   
                          YUM     139.919998  139.919998  140.020004   
                          ZBH     108.220001  108.220001  110.190002   
                          ZBRA    368.600006  368.600006  374.029999   
                          ZTS     194.349457  194.809998  197.509995   

Price                                    low        open       volume  \
date                      ticker                                        
2014-10-03 00:00:00+00:00 A        40.171673   40.221745    2205065.0   
                          AAPL     24.760000   24.860001  173878400.0   
                          ABBV     57.340000   57.419998   11675600.0   
                          ABT      41.279999   41.410000    3430900.0   
                          ACGL     18.320000   18.333332    1470300.0   
...                                      ...         ...          ...   
2024-09-27 00:00:00+00:00 XYL     133.779999  134.660004     691300.0   
                          YUM     136.990005  137.520004    2778300.0   
                          ZBH     108.190002  108.769997    1166200.0   
                          ZBRA    368.269989  372.299988     213600.0   
                          ZTS     194.419998  195.649994    1343700.0   

Price                             garman_klass_vol        rsi    bb_low  \
date                      ticker                                          
2014-10-03 00:00:00+00:00 A              -0.001745        NaN       NaN   
                          AAPL           -0.005326        NaN       NaN   
                          ABBV           -0.061293        NaN       NaN   
                          ABT            -0.013375        NaN       NaN   
                          ACGL            0.000021        NaN       NaN   
...                                            ...        ...       ...   
2024-09-27 00:00:00+00:00 XYL             0.000172  52.810998  4.849547   
                          YUM             0.000124  62.536185  4.867805   
                          ZBH             0.000158  48.599342  4.625674   
                          ZBRA            0.000082  64.223784  5.759747   
                          ZTS             0.000107  61.511356  5.218571   

Price                             bb_middle   bb_high       atr      macd  \
date                      ticker                                            
2014-10-03 00:00:00+00:00 A             NaN       NaN       NaN       NaN   
                          AAPL          NaN       NaN       NaN       NaN   
                          ABBV          NaN       NaN       NaN       NaN   
                          ABT           NaN       NaN       NaN       NaN   
                          ACGL          NaN       NaN       NaN       NaN   
...                                     ...       ...       ...       ...   
2024-09-27 00:00:00+00:00 XYL      4.891114  4.932681  0.935011  0.088067   
                          YUM      4.902218  4.936632  1.023418  0.131712   
                          ZBH      4.687395  4.749115 -0.623002 -0.377126   
                          ZBRA     5.851378  5.943010  0.350878  1.174472   
                          ZTS      5.256497  5.294424  0.774667  1.035597   

Price                             dollar_volume  
date                      ticker                 
2014-10-03 00:00:00+00:00 A        

### 3.Aggregate to monthly level and filter top 50 most liquid stocks for each month 

--to reduce training time and experiment with features and strategies , we convert the business-daily data to month-end frequency

df.unstack('ticker'):

This reshapes the DataFrame by "unstacking" the 'ticker' level of the index, which moves it from being a row index to being a column index.
This will produce a wide-format DataFrame where each unique 'ticker' value becomes a column header, and all other index levels are maintained.
['dollar_volume']:

After unstacking, this selects the 'dollar_volume' column (or MultiIndex level), focusing only on this column for further operations.

.resample('M'):
This resamples the data to a monthly frequency (using 'M' for monthly).
The resample method is typically used for time series data and will create a new index with the first date of each month.

.mean():
This calculates the mean of 'dollar_volume' for each ticker within each monthly period.
The result is a DataFrame with a monthly time index and each ticker as a separate column, where the values are the mean of 'dollar_volume' for each ticker in each month.

.stack('ticker'):
This "stacks" the ticker level back from being a column index to being part of the row index, returning to a long-format DataFrame.
After stacking, the DataFrame will have a MultiIndex with date (monthly) and ticker levels.

.to_frame('dollar_volume'):
Converts the Series back to a DataFrame and renames the column to 'dollar_volume'.
This step is often used when a single-column DataFrame is preferred for consistent structure or further manipulation.

In [13]:
df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume')

dollar_volume
date                      ticker               
2014-10-31 00:00:00+00:00 A          130.132314
                          AAPL      5404.400113
                          ABBV       790.715442
                          ABT        195.698929
                          ACGL        23.590734
...                                         ...
2024-09-30 00:00:00+00:00 XYL        160.566119
                          YUM        287.368688
                          ZBH        193.110143
                          ZBRA       121.973536
                          ZTS        368.349908

[58555 rows x 1 columns]

In [15]:
# technical indicators column:

last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume','volume','open','close','high','low']]
last_cols


#now we would concetanate the dollar volume with rest of the indicators:
data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
           df.unstack()[last_cols].resample('M').last().stack('ticker')],axis=1)).dropna()

data

dollar_volume   adj close  garman_klass_vol  \
date                      ticker                                                
2014-11-30 00:00:00+00:00 A          112.969374   39.399696         -0.002514   
                          AAPL      4366.292581   26.485039         -0.005361   
                          ABBV       273.920907   45.751591         -0.065179   
                          ABT        152.154585   36.786415         -0.013803   
                          ACGL        27.891787   19.106667          0.000021   
...                                         ...         ...               ...   
2024-09-30 00:00:00+00:00 XYL        160.566119  134.509995          0.000172   
                          YUM        287.368688  139.919998          0.000124   
                          ZBH        193.110143  108.220001          0.000158   
                          ZBRA       121.973536  368.600006          0.000082   
                          ZTS        368.349908  194.349457          0.000107   

                                        rsi    bb_low  bb_middle   bb_high  \
date                      ticker                                             
2014-11-30 00:00:00+00:00 A       65.857326  3.626446   3.666908  3.707369   
                          AAPL    79.936425  3.195682   3.263211  3.330740   
                          ABBV    74.712855  3.715873   3.781591  3.847309   
                          ABT     62.980189  3.601911   3.618318  3.634725   
                          ACGL    59.646062  2.986044   2.997348  3.008653   
...                                     ...       ...        ...       ...   
2024-09-30 00:00:00+00:00 XYL     52.810998  4.849547   4.891114  4.932681   
                          YUM     62.536185  4.867805   4.902218  4.936632   
                          ZBH     48.599342  4.625674   4.687395  4.749115   
                          ZBRA    64.223784  5.759747   5.851378  5.943010   
                          ZTS     61.511356  5.218571   5.256497  5.294424   

                                       atr      macd  
date                      ticker                      
2014-11-30 00:00:00+00:00 A      -1.121415  0.346492  
                          AAPL   -1.022122  0.198582  
                          ABBV   -1.075526  0.864662  
                          ABT    -1.148236  0.241693  
                          ACGL   -1.227769 -0.112052  
...                                    ...       ...  
2024-09-30 00:00:00+00:00 XYL     0.935011  0.088067  
                          YUM     1.023418  0.131712  
                          ZBH    -0.623002 -0.377126  
                          ZBRA    0.350878  1.174472  
                          ZTS     0.774667  1.035597  

[58048 rows x 9 columns]

**calculate 5 year rolling average of dollar volume for each stocks before filtering**

In [16]:
data['dollar_volume']=(data['dollar_volume'].unstack('ticker').rolling(5*12).mean().stack())

data['dollar_vol_rank']=(data.groupby('date')['dollar_volume'].rank(ascending=False))

data = data[data['dollar_vol_rank']<150]

data

#we have calculate the monthly values for all the features 


dollar_volume   adj close  garman_klass_vol  \
date                      ticker                                                
2019-10-31 00:00:00+00:00 AAPL      4865.184953   60.111645          0.000900   
                          ABBV       410.910315   64.195099         -0.018022   
                          ABT        305.309014   76.628838         -0.002921   
                          ACN        263.728003  172.285400         -0.002118   
                          ADBE       457.344572  277.929993          0.000104   
...                                         ...         ...               ...   
2024-09-30 00:00:00+00:00 VZ         776.954031   44.200008          0.000025   
                          WFC        990.058214   55.579750         -0.000026   
                          WMT       1037.630357   79.779999          0.000036   
                          XOM       1538.960969  115.820000          0.000182   
                          ZTS        337.073865  194.349457          0.000107   

                                        rsi    bb_low  bb_middle   bb_high  \
date                      ticker                                             
2019-10-31 00:00:00+00:00 AAPL    68.908128  4.005269   4.066120  4.126971   
                          ABBV    70.480816  4.068833   4.129044  4.189254   
                          ABT     54.403508  4.288739   4.325429  4.362118   
                          ACN     47.064275  5.132833   5.152503  5.172172   
                          ADBE    51.985924  5.570602   5.610361  5.650119   
...                                     ...       ...        ...       ...   
2024-09-30 00:00:00+00:00 VZ      65.041478  3.726660   3.781470  3.836280   
                          WFC     51.187903  3.956308   4.021197  4.086086   
                          WMT     63.358678  4.345042   4.380543  4.416044   
                          XOM     51.821142  4.706781   4.745506  4.784230   
                          ZTS     61.511356  5.218571   5.256497  5.294424   

                                       atr      macd  dollar_vol_rank  
date                      ticker                                       
2019-10-31 00:00:00+00:00 AAPL   -0.592924  0.515936              1.0  
                          ABBV   -0.941640  0.762890             58.0  
                          ABT    -0.009821  0.041810             83.0  
                          ACN    -0.348493 -0.500077            101.0  
                          ADBE   -0.387039 -0.302571             49.0  
...                                    ...       ...              ...  
2024-09-30 00:00:00+00:00 VZ     -0.326941  1.906847             53.0  
                          WFC     1.731254 -0.238594             37.0  
                          WMT     2.010067  2.928695             33.0  
                          XOM     1.213081 -0.132851             16.0  
                          ZTS     0.774667  1.035597            147.0  

[8940 rows x 10 columns]

**4.calculate monthly returns for different time horizons as features**

-- To capture time series dynamics that reflect ,for example momentum patterns ,we compute historical returns using method .pct_change(lag), that is returns over various monthly periods as indentified by lags.

In [17]:
def calculate_returns(df):
    outlier_cutoff=0.005
    lags=[1,2,3,6,9,12]
    
    
    
    for lag in lags:
        
        df[f'return_{lag}m']=(df['adj close']
                              .pct_change(lag)
                              .pipe(lambda x : x.clip(lower=x.quantile(outlier_cutoff),upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df

# .pipe(lambda x: ...): This is a functional approach to modify the data.
# It allows you to apply a function (in this case, a lambda function) to the Series.
# x.clip(...): Clipping limits the values of the Series. The lower and upper parameters specify the bounds.
# lower=x.quantile(outlier_cutoff): The lower bound is set to the value at the specified quantile, 
# defined by outlier_cutoff. This means any values below this quantile will be replaced with this value.
# upper=x.quantile(1-outlier_cutoff): The upper bound is similarly set to the value at the upper quantile.

data = data.groupby(level=1).apply(calculate_returns).dropna()
data

dollar_volume   adj close  \
ticker date                      ticker                              
AAPL   2020-10-31 00:00:00+00:00 AAPL      6366.229547  106.265976   
       2020-11-30 00:00:00+00:00 AAPL      6501.032358  116.413635   
       2020-12-31 00:00:00+00:00 AAPL      6649.971148  129.751541   
       2021-01-31 00:00:00+00:00 AAPL      6807.031112  129.037735   
       2021-02-28 00:00:00+00:00 AAPL      6953.163432  118.751892   
...                                                ...         ...   
XOM    2024-06-30 00:00:00+00:00 XOM       1488.793795  114.200592   
       2024-07-31 00:00:00+00:00 XOM       1504.596015  117.642876   
       2024-08-31 00:00:00+00:00 XOM       1518.330587  117.940002   
       2024-09-30 00:00:00+00:00 XOM       1538.960969  115.820000   
ZTS    2024-09-30 00:00:00+00:00 ZTS        337.073865  194.349457   

                                         garman_klass_vol        rsi  \
ticker date                      ticker                                
AAPL   2020-10-31 00:00:00+00:00 AAPL            0.000003  43.770338   
       2020-11-30 00:00:00+00:00 AAPL            0.000604  54.008260   
       2020-12-31 00:00:00+00:00 AAPL           -0.000159  61.780727   
       2021-01-31 00:00:00+00:00 AAPL            0.000181  51.602268   
       2021-02-28 00:00:00+00:00 AAPL            0.000049  36.515081   
...                                                   ...        ...   
XOM    2024-06-30 00:00:00+00:00 XOM             0.000049  53.431099   
       2024-07-31 00:00:00+00:00 XOM            -0.000059  57.851750   
       2024-08-31 00:00:00+00:00 XOM             0.000072  54.346519   
       2024-09-30 00:00:00+00:00 XOM             0.000182  51.821142   
ZTS    2024-09-30 00:00:00+00:00 ZTS             0.000107  61.511356   

                                           bb_low  bb_middle   bb_high  \
ticker date                      ticker                                  
AAPL   2020-10-31 00:00:00+00:00 AAPL    4.683823   4.742640  4.801457   
       2020-11-30 00:00:00+00:00 AAPL    4.693996   4.746338  4.798679   
       2020-12-31 00:00:00+00:00 AAPL    4.762355   4.834837  4.907318   
       2021-01-31 00:00:00+00:00 AAPL    4.799064   4.875032  4.951000   
       2021-02-28 00:00:00+00:00 AAPL    4.788495   4.865137  4.941778   
...                                           ...        ...       ...   
XOM    2024-06-30 00:00:00+00:00 XOM     4.685024   4.724820  4.764615   
       2024-07-31 00:00:00+00:00 XOM     4.708335   4.747499  4.786663   
       2024-08-31 00:00:00+00:00 XOM     4.737298   4.767880  4.798463   
       2024-09-30 00:00:00+00:00 XOM     4.706781   4.745506  4.784230   
ZTS    2024-09-30 00:00:00+00:00 ZTS     5.218571   5.256497  5.294424   

                                              atr      macd  dollar_vol_rank  \
ticker date                      ticker                                        
AAPL   2020-10-31 00:00:00+00:00 AAPL    1.354647 -0.610562              2.0   
       2020-11-30 00:00:00+00:00 AAPL    0.667135 -0.091531              2.0   
       2020-12-31 00:00:00+00:00 AAPL    0.768531  1.579339              2.0   
       2021-01-31 00:00:00+00:00 AAPL    1.401473  1.118208              2.0   
       2021-02-28 00:00:00+00:00 AAPL    1.097911 -1.511428              2.0   
...                                           ...       ...              ...   
XOM    2024-06-30 00:00:00+00:00 XOM     0.504451 -0.339191             15.0   
       2024-07-31 00:00:00+00:00 XOM     0.666414  0.760932             16.0   
       2024-08-31 00:00:00+00:00 XOM     0.664306  0.414077             16.0   
       2024-09-30 00:00:00+00:00 XOM     1.213081 -0.132851             16.0   
ZTS    2024-09-30 00:00:00+00:00 ZTS     0.774667  1.035597            147.0   

                                         return_1m  return_2m  return_3m  \
ticker date                      ticker                                    
AAPL   2020-10-31 00:00:00+00:00 AAPL

**5. Download fama-french factors and calculate rolling factor betas**

--we will introduce fema french data to estimate the exposure of assests to common risk factors using linear regression 
--The five fema-french factors, namely market risk , value ,operating profitability and investment have been shown emperically to explain asset returns are commonly used to assess risk/return profile of portfolios. Hence,it is natural to include past factor exposures as financial features in models.
--we can access the historical factor returns using the pandas-datareader and estimate historical exposure using rollingOLS (rolling linear regression) 

In [48]:
import pandas as pd
import pandas_datareader.data as web

factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                               'famafrench',
                               start='2010')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('M').last().div(100)

factor_data.index.name = 'date'

factor_data = factor_data.join(data['return_1m']).sort_index()

factor_data

,Mkt-RF,SMB,HML,RMW,CMA,return_1m
date,,,,,,
2010-01-31,-0.0336,0.0034,0.0043,-0.0127,0.0046,0.00
2010-02-28,0.0340,0.0151,0.0322,-0.0027,0.0142,0.01
2010-03-31,0.0631,0.0185,0.0221,-0.0065,0.0169,0.02
2010-04-30,0.0200,0.0498,0.0289,0.0069,0.0172,0.03
2010-05-31,-0.0789,0.0005,-0.0244,0.0130,-0.0022,0.04
...,...,...,...,...,...,...
2024-05-31,0.0434,0.0076,-0.0166,0.0298,-0.0307,NaN
2024-06-30,0.0277,-0.0437,-0.0331,0.0051,-0.0178,NaN
2024-07-31,0.0124,0.0828,0.0573,0.0022,0.0043,NaN


--filter out stocks with less than 10 months of data 